In [1]:





import torch
import torch.nn as nn
import torchvision

# Load observations from the mnist dataset. The observations are divided into a training set and a test set
mnist_train = torchvision.datasets.FashionMNIST('./data', train=True, download=True)
x_train = mnist_train.data.reshape(-1, 1, 28, 28).float()  # torch.functional.nn.conv2d argument must include channels (1)
y_train = torch.zeros((mnist_train.targets.shape[0], 10))  # Create output tensor
y_train[torch.arange(mnist_train.targets.shape[0]), mnist_train.targets] = 1  # Populate output

mnist_test = torchvision.datasets.FashionMNIST('./data', train=False, download=True)
x_test = mnist_test.data.reshape(-1, 1, 28, 28).cuda().float()  # torch.functional.nn.conv2d argument must include channels (1)
y_test = torch.zeros((mnist_test.targets.shape[0], 10)).cuda()  # Create output tensor
y_test[torch.arange(mnist_test.targets.shape[0]), mnist_test.targets] = 1  # Populate output

# Normalization of inputs
mean = x_train.mean()
std = x_train.std()
x_train = (x_train - mean) / std
x_test = (x_test - mean) / std

# Divide training data into batches to speed up optimization
batches = 600
x_train_batches = torch.split(x_train.cuda(), batches)
y_train_batches = torch.split(y_train.cuda(), batches)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


/home/sigmund/Documents/skole/sem5/machinelearning/test/venv/lib/python3.9/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [2]:
class ConvolutionalNeuralNetworkModel(nn.Module):
    def __init__(self):
        super(ConvolutionalNeuralNetworkModel, self).__init__()

        # Two sequential layers
        # Model layers (includes initialized model variables):
        self.logits1 = nn.Sequential(nn.Conv2d(1, 32, kernel_size=3, padding=1),
                                    nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
                                    ).cuda()
        self.logits2 = nn.Sequential(nn.Conv2d(32, 64, kernel_size=3),
                                    nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
                                    ).cuda()


        self.lin1 = nn.Linear(64 * 6 * 6, 600).cuda()
        self.drop = nn.Dropout2d(0.25)
        self.lin2 = nn.Linear(600, 124).cuda()
        self.lin3 = nn.Linear(124, 10).cuda()

    def logits(self, x):
        out = self.logits1(x)
        out = self.logits2(out)
        out = out.view(out.size(0), -1)
        out = self.lin1(out)
        out = self.drop(out)
        out = self.lin2(out)
        out = self.lin3(out)
        return out

    # Predictor
    def f(self, x):
        return torch.softmax(self.logits(x), dim=1)

    # Cross Entropy loss
    def loss(self, x, y):
        return nn.functional.cross_entropy(self.logits(x), y.argmax(1))

    # Accuracy
    def accuracy(self, x, y):
        return torch.mean(torch.eq(self.f(x).argmax(1), y.argmax(1)).float())

model = ConvolutionalNeuralNetworkModel()

# Optimize: adjust W and b to minimize loss using stochastic gradient descent
optimizer = torch.optim.Adam(model.parameters(), 0.001)
for epoch in range(20):
    for batch in range(len(x_train_batches)):
        model.loss(x_train_batches[batch], y_train_batches[batch]).backward()  # Compute loss gradients
        optimizer.step()  # Perform optimization by adjusting W and b,
        optimizer.zero_grad()  # Clear gradients for next step
    print("epoch: %s accuracy = %s" % (epoch, model.accuracy(x_test, y_test)))


print(model)

/home/sigmund/Documents/skole/sem5/machinelearning/test/venv/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


epoch: 0 accuracy = tensor(0.8612, device='cuda:0')
epoch: 1 accuracy = tensor(0.8659, device='cuda:0')
epoch: 2 accuracy = tensor(0.8916, device='cuda:0')
epoch: 3 accuracy = tensor(0.8972, device='cuda:0')
epoch: 4 accuracy = tensor(0.9008, device='cuda:0')
epoch: 5 accuracy = tensor(0.9018, device='cuda:0')
epoch: 6 accuracy = tensor(0.9025, device='cuda:0')
epoch: 7 accuracy = tensor(0.9047, device='cuda:0')
epoch: 8 accuracy = tensor(0.9057, device='cuda:0')
epoch: 9 accuracy = tensor(0.9051, device='cuda:0')
epoch: 10 accuracy = tensor(0.9102, device='cuda:0')
epoch: 11 accuracy = tensor(0.9102, device='cuda:0')
epoch: 12 accuracy = tensor(0.9067, device='cuda:0')
epoch: 13 accuracy = tensor(0.9093, device='cuda:0')
epoch: 14 accuracy = tensor(0.9072, device='cuda:0')
epoch: 15 accuracy = tensor(0.9104, device='cuda:0')
epoch: 16 accuracy = tensor(0.9093, device='cuda:0')
epoch: 17 accuracy = tensor(0.9058, device='cuda:0')
epoch: 18 accuracy = tensor(0.9070, device='cuda:0')
epo

In [3]:
class ConvolutionalNeuralNetworkModel(nn.Module):
    def __init__(self):
        super(ConvolutionalNeuralNetworkModel, self).__init__()

        # Two sequential layers
        # Model layers (includes initialized model variables):
        self.logits1 = nn.Sequential(nn.Conv2d(1, 32, kernel_size=5, padding=2),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size=2)
                                    ).cuda()
        self.logits2 = nn.Sequential(nn.Conv2d(32, 64, kernel_size=5, padding=2),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size=2),
                                    nn.Flatten()).cuda()

        self.lin1 = nn.Linear(64 * 7 * 7, 600).cuda()
        self.lin3 = nn.Linear(600, 10).cuda()

    def logits(self, x):
        out = self.logits1(x)
        out = self.logits2(out)
        out = out.view(out.size(0), -1)
        out = self.lin1(out)
        out = self.lin3(out)
        return out

    # Predictor
    def f(self, x):
        return torch.softmax(self.logits(x), dim=1)

    # Cross Entropy loss
    def loss(self, x, y):
        return nn.functional.cross_entropy(self.logits(x), y.argmax(1))

    # Accuracy
    def accuracy(self, x, y):
        return torch.mean(torch.eq(self.f(x).argmax(1), y.argmax(1)).float())

model = ConvolutionalNeuralNetworkModel()

# Optimize: adjust W and b to minimize loss using stochastic gradient descent
optimizer = torch.optim.Adam(model.parameters(), 0.001)
for epoch in range(20):
    for batch in range(len(x_train_batches)):
        model.loss(x_train_batches[batch], y_train_batches[batch]).backward()  # Compute loss gradients
        optimizer.step()  # Perform optimization by adjusting W and b,
        optimizer.zero_grad()  # Clear gradients for next step

    print("epoch: %s accuracy = %s" % (epoch, model.accuracy(x_test, y_test)))


print(model)


epoch: 0 accuracy = tensor(0.8705, device='cuda:0')
epoch: 1 accuracy = tensor(0.8889, device='cuda:0')
epoch: 2 accuracy = tensor(0.9015, device='cuda:0')
epoch: 3 accuracy = tensor(0.9037, device='cuda:0')
epoch: 4 accuracy = tensor(0.9055, device='cuda:0')
epoch: 5 accuracy = tensor(0.9069, device='cuda:0')
epoch: 6 accuracy = tensor(0.9080, device='cuda:0')
epoch: 7 accuracy = tensor(0.9106, device='cuda:0')
epoch: 8 accuracy = tensor(0.9057, device='cuda:0')
epoch: 9 accuracy = tensor(0.9020, device='cuda:0')
epoch: 10 accuracy = tensor(0.9079, device='cuda:0')
epoch: 11 accuracy = tensor(0.9125, device='cuda:0')
epoch: 12 accuracy = tensor(0.9113, device='cuda:0')
epoch: 13 accuracy = tensor(0.9110, device='cuda:0')
epoch: 14 accuracy = tensor(0.9127, device='cuda:0')
epoch: 15 accuracy = tensor(0.9125, device='cuda:0')
epoch: 16 accuracy = tensor(0.9110, device='cuda:0')
epoch: 17 accuracy = tensor(0.9102, device='cuda:0')
epoch: 18 accuracy = tensor(0.9055, device='cuda:0')
epo